In [ ]:
import tensorflow as tf

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [ ]:
input_width=28
input_height=28
input_channels=1
input_size=784

In [5]:
mnist.train.images.shape

(55000, 784)

In [ ]:
n_conv1=32           #No of units in convolution layer 1 
n_conv2=64           #No of units in convolution layer 2
n_dense=1024         #No of units in dense layer
n_out=10             #No of units in the output layer

In [7]:
k1=5             #FILTER SIZE FOR CONVOLUTION LAYER 1
k2=5             #FILTER SIZE FOR CONVOLUTION LAYER 2
stride1=1
stride2=1
pool1_k=2        #Max_Pool filter size 
pool2_k=2        #for both the max pool layers
pool2_output_size = (input_width//(pool1_k*pool2_k))*(input_height//(pool1_k*pool2_k))*n_conv2
pool2_output_size      #This is basically the input features that go to the dense(hidden) layer with 1024 units

3136

In [8]:
weights={
    "wc1":tf.Variable(tf.random_normal([k1,k1,input_channels,n_conv1])),
    "wc2":tf.Variable(tf.random_normal([k2,k2,n_conv1,n_conv2])),
    "wh1":tf.Variable(tf.random_normal([pool2_output_size,n_dense])),
    "wo":tf.Variable(tf.random_normal([n_dense,n_out]))
}

biases={
    "bc1":tf.Variable(tf.random_normal([n_conv1])),
    "bc2":tf.Variable(tf.random_normal([n_conv2])),
    "bh1":tf.Variable(tf.random_normal([n_dense])),
    "bo":tf.Variable(tf.random_normal([n_out]))
}

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
def cnn(x,weights,biases,keep_prob=0.75 , training = True):
    images=tf.reshape(x,shape=[-1,input_height,input_width,input_channels])           #Reshaping the images 
    conv1=conv(images,weights['wc1'],biases['bc1'],stride1)
    conv1_pool=maxpooling(conv1,pool1_k)

    conv2=conv(conv1_pool,weights['wc2'],biases['bc2'],stride2)
    conv2_pool=maxpooling(conv2,pool2_k)

    dense_input=tf.reshape(conv2_pool,shape=[-1,pool2_output_size])
    dense_output_before_activation=tf.add(tf.matmul(dense_input,weights['wh1']),biases['bh1'])
    dense_output = tf.nn.relu(dense_output_before_activation)
    if training:
        dropout_output = tf.nn.dropout(dense_output , keep_prob)
    else:
        dropout_output = dense_output
    output = tf.add(tf.matmul(dropout_output,weights['wo']),biases['bo'])
    return output

In [ ]:
#Now in the first step we move the filter on the images using the functiom tf.nn.conv2d
#Then we add the biases using a special function tf.nn.bias_add
#Then in the third step we apply the activation function and return the output
def conv(images , weights , biases, stride=1):
    out = tf.nn.conv2d(images,weights,padding="SAME",strides = [1,stride,stride,1])
    out = tf.nn.bias_add(out,biases)
    out=tf.nn.relu(out)
    return out

In [ ]:
def maxpooling(images, k = 2):
    return tf.nn.max_pool(images,ksize=[1,k,k,1],padding="SAME",strides=[1,k,k,1])

In [12]:
x=tf.placeholder("float",[None,input_size])
y=tf.placeholder(tf.int32,[None,n_out])

pred=cnn(x,weights,biases)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))

In [ ]:
optimizer=tf.train.AdamOptimizer(learning_rate=0.01)
optimize=optimizer.minimize(cost)

In [ ]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [17]:
batch_size=100
for i in range(25):
    num_batches=int(mnist.train.num_examples/batch_size)
    total_cost=0
    for j in range(num_batches):
        batch_x,batch_y=mnist.train.next_batch(batch_size)     #Tensorflow already provides an inbuilt function to create batches of x and y according to the batch size
        c,_=sess.run([cost,optimize],feed_dict={x:batch_x,y:batch_y})
        total_cost+=c
    print(total_cost)

973457.2251079082
36500.75811350427
20181.06441855134
11895.142187070101
9067.66943077163
8404.675237813295
7338.042391553818
7007.675895127662
6471.650135979042
5712.759632501227
5111.3736998966515
5028.181658680714
4271.106277514969
2911.8879490922945
3914.0710258658364
4118.541610621545
2994.3174131283295
2806.698722378329
2183.884401855379
2603.5675622801623
2337.423989041373
2464.2448781751127
2314.554405799908
1701.8258978822423
1677.688647805747


In [27]:
predictions = tf.argmax(pred,1)
true_labels = tf.argmax(y,1)
correct_predictions=tf.equal(predictions,true_labels)
predictions_eval,labels,correct_pred = sess.run([predictions,true_labels,correct_predictions], feed_dict={x:mnist.train.images,y:mnist.train.labels})
predictions_eval,labels,correct_pred

(array([1, 9, 7, ..., 4, 6, 1]),
 array([1, 9, 7, ..., 4, 6, 1]),
 array([ True,  True,  True, ...,  True,  True,  True]))

In [28]:
correct_pred.sum()

54672

In [29]:
#Accuracy obtained 
54672/55000 

0.9940363636363636